In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

x= torch.randn(1).cuda()
print(x)

1.13.0
True
1
cuda
tensor([-0.9516], device='cuda:0')


In [2]:
data_path = './data'
#transform to tensor and normalize
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

batch_size = 2

#download trainset
cifar10 = torchvision.datasets.CIFAR10(data_path,train=True,download =True,transform = transform)

#setup trainloader
cifar10_loader = torch.utils.data.DataLoader(cifar10, batch_size=batch_size,shuffle=True)

#download testset
cifar10_val = torchvision.datasets.CIFAR10(data_path,train=False,download =True,transform = transform)

#setup testloader
cifar10_val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=batch_size,shuffle=False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')





Files already downloaded and verified
Files already downloaded and verified


In [3]:
# model P1A

 #used 3072 since one pic is 32 x 32 res, and 3 pixel coloration inputs per pixel, 32 x 32 x 3 = 3072
model = nn.Sequential(\
                 nn.Linear(3072,512),\
                 nn.Tanh(),\
                 nn.Linear(512,10),\
                 nn.LogSoftmax(dim=1))


model = model.to(device)
#LOSS
criterion = nn.CrossEntropyLoss() 
#OPTIMIZER
#optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=0.003) 
optimizer = optim.SGD(model.parameters(),lr=0.000560114, momentum=0.9) 



In [4]:
def softmax(x):
    return torch.exp(x)/torch.exp(x).sum() # for 0.09, e^1/(e^1 + e^2 + e^3) where top is 1
#x = torch.tensor([1.0,2.0,3.0])

#softmax(x)
#imgs, labels = cifar10_loader
#print(imgs.size(0))

In [5]:
def training(n_epoch):
    t0 = time.time()
    #torch.cuda.empty_cache()
    for epoch in range(n_epoch):
        
       
        
        for imgs, labels in cifar10_loader:
            imgs,labels = imgs.to(device),labels.to(device)
            batch_size = imgs.shape[0] #takes image data, puts into 0 x data tensor
            outputs = model(imgs.view(batch_size, -1)) #unsqueeze(0) adds dimensionality at dimension 0, (0,1)--> (0,1,2) where 1 is the new dimension
            loss = criterion(outputs,labels)
            
            optimizer.zero_grad()
            #torch.cuda.empty_cache()
            loss.backward()
            optimizer.step()
        print(" Epoch: %d, Loss: %F" % (epoch, float(loss)))
    print(f'training time = {time.time()-t0}')
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in cifar10_val_loader:
            imgs,labels = imgs.to(device),labels.to(device)
            batch_size = imgs.shape[0]
            outputs = model(imgs.view(batch_size,-1))
            _,predicted = torch.max(outputs,dim=1)
            total += labels.shape[0] #create 
            correct += int((predicted == labels).sum()) # if predicted is the answer, add it to sum
    print(correct)
    print(total)
    print("Accuracy: %f", correct / total)
    
#training(3)
#note, this is really bad, accuracy = 38.47%,possibly also wasn't learning

In [6]:
#problem 2a
model = nn.Sequential(\
                 nn.Linear(3072,1024),\
                 nn.LeakyReLU(),\
                 nn.Linear(1024,512),\
                 nn.LeakyReLU(),\
                 nn.Linear(512,128),\
                 nn.LeakyReLU(),\
                 nn.Linear(128,10)\
                     )


model = model.to(device)
#training

In [7]:
def training_cnn(n_epoch):
    t0 = time.time()
    #torch.cuda.empty_cache()
    for epoch in range(n_epoch):
        
       
        
        for imgs, labels in cifar10_loader:
            imgs,labels = imgs.to(device),labels.to(device)
            batch_size = imgs.shape[0] #takes image data, puts into 0 x data tensor
            outputs = model(imgs) #unsqueeze(0) adds dimensionality at dimension 0, (0,1)--> (0,1,2) where 1 is the new dimension
            loss = criterion(outputs,labels)                 # [[1,2,3]] --> [[[1],[2],[3]]]
            
            optimizer.zero_grad()
            #torch.cuda.empty_cache()
            loss.backward()
            optimizer.step()
        print(" Epoch: %d, Loss: %F" % (epoch, float(loss)))
    print(f'training time = {time.time()-t0}')
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in cifar10_val_loader:
            imgs,labels = imgs.to(device),labels.to(device)
            batch_size = imgs.shape[0]
            outputs = model(imgs)
            _,predicted = torch.max(outputs,dim=1)
            total += labels.shape[0] #create 
            correct += int((predicted == labels).sum()) # if predicted is the answer, add it to sum
    print(correct)
    print(total)
    print("Accuracy: %f", correct / total)

In [8]:
#Problem 1b
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6, 5) #3,16,kernel_size=3,padding=1
        self.conv2 = nn.Conv2d(6,16,5) #16,8,kernel_size=3,padding=1
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,16)
        self.fc3 = nn.Linear(16,10)
    def forward(self,x):
        out = F.max_pool2d(F.relu(self.conv1(x)),2)
        out = F.max_pool2d(F.relu(self.conv2(out)),2)
        out = out.view(-1, 16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
model = model()
model = model.to(device)

training_cnn(300)# YOU BUFFOON, YOU ONLY FOUND THE BEST STRUCTURE FOR 10 PICTURES, NOT THE 50,000 SAMPLES

 Epoch: 0, Loss: 2.395412
 Epoch: 1, Loss: 2.289831


KeyboardInterrupt: 